In [1]:
import findspark
findspark.init()
import tweepy
import string
import os
import re
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizer
#consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
#consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
#access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
#access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
#auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth)


In [2]:
#open 2 command prompts
#terminal 1
#cd %SPARK_HOME%
#bin\spark-class2.cmd org.apache.spark.deploy.master.Master
#open in browser localhost:8080
#terminal 2
#cd %SPARK_HOME%
#bin\spark-class2.cmd org.apache.spark.deploy.worker.Worker -c 4 -m 32G spark://192.168.1.227:7077
#open in browser localhost:8081
spark = SparkSession.builder.master('spark://192.168.1.227:7077').appName('classifier').config('spark.executor.memory', '32g').getOrCreate()
df = spark.read.options(delimiter=",",header=False).csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df=df.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df2=spark.read.options(delimiter=",",header=False).csv("trainingandtestdata/testdata.manual.2009.06.14.csv")
df2=df2.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')

In [3]:
from pyspark.sql.types import StringType
def preprocessing(text):
    # process the tweets

    #Convert to lower case
    tweet = text.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+',' ',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
tem=df.union(df2)
brand_udf=udf(preprocessing,StringType())
tem=tem.withColumn('text',brand_udf(tem['Tweet']))

In [4]:
from pyspark.sql.types import IntegerType
tem1=tem.select('tweet_id','Tweet','text','polarity')
tem1 = tem1.withColumn('polarity', tem1['polarity'].cast(IntegerType()))
(trainingData, testData) = tem1.randomSplit([0.8, 0.2])

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures")
idf = IDF(minDocFreq=2, inputCol="rawFeatures", outputCol="features")

lr = LogisticRegression(labelCol='polarity')

# Pipeline Architecture
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [6]:
predictions = model.transform(testData)
predictions1=predictions.toPandas()
# Select example rows to display.
#predictions.select("text", "polarity", "prediction").show(5,False)

In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="polarity", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.752789071140396


In [8]:
import pandas as pd
import json
testData=testData.toPandas()
predictions=predictions1
testData['prediction']=predictions['prediction']
testData=testData[['tweet_id','Tweet','polarity','prediction']]
spark.stop()

In [9]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["tweet"]
records = json.loads(testData.T.to_json()).values()
mydb.mycol.insert_many(records)

In [28]:
spark= SparkSession.builder.appName("Twitter Streaming App").getOrCreate()
tweet_df = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 5556).load()
tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")
tweet_df_string = tweet_df_string.withColumnRenamed("value" , "tweet")
writeTweet = tweet_df_string.writeStream.format("csv").option("format" , "append").option("path", "C:/Users/Sajin.LAPTOP-RE0DL8PH/Sajin/Shajee_Fall_2021/CS644/temp").option("checkpointLocation" , "checkpoints").queryName("tweetquery").start()

In [29]:
import time
time.sleep(60) 
writeTweet.stop()
import pandas as pd
import os
import glob
import pandas as pd
os.chdir("C:/Users/Sajin.LAPTOP-RE0DL8PH/Sajin/Shajee_Fall_2021/CS644/temp")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
dataframe = pd.DataFrame()
for i in all_filenames:
    if (os.stat(f"C:/Users/Sajin.LAPTOP-RE0DL8PH/Sajin/Shajee_Fall_2021/CS644/temp/{i}").st_size != 0):
        try:
            d = pd.read_csv(f"C:/Users/Sajin.LAPTOP-RE0DL8PH/Sajin/Shajee_Fall_2021/CS644/temp/{i}" , engine="python")
            dt = d.T
            dt = dt.reset_index()
            dataframe = dataframe.append(dt)
        except:
            pass
dataframe = dataframe.rename(columns={"index" : "Tweet"})
dataframe = dataframe.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe[dataframe['Tweet'] != ""]
dataframe = dataframe[dataframe['Tweet'] != "."]
dataframe = dataframe[dataframe['Tweet'] != None]

dataframe = dataframe.reset_index(drop=True)
range1 = range(100, 100 + dataframe['Tweet'].size  )
list1 = list(range1)
dfg = pd.DataFrame(list1 , columns = ['tweet_id'])
dfg = dfg['tweet_id'].astype('str')
list_polarity =[]
for b in range(dataframe['Tweet'].size):
    list_polarity.append(0)
    
dfp = pd.DataFrame(list_polarity , columns = ['polarity'])
dataframe = dataframe.join(dfg)
dataframe = dataframe.join(dfp)
sparkDF = spark.createDataFrame(dataframe)
sparkDF=sparkDF[['tweet_id','Tweet','polarity']]
brand_udf=udf(preprocessing,StringType())
sparkDF=sparkDF.withColumn('text',brand_udf(sparkDF['Tweet']))

Empty DataFrame
Columns: []
Index: []


KeyError: 'Tweet'

In [ ]:
pred = model.transform(sparkDF)
pred2=pred.toPandas()
spark.stop()
pred2=pred2[['tweet_id','Tweet','prediction']]
mycol2 = mydb["tweet2"]
records2 = json.loads(pred2.T.to_json()).values()
mydb.mycol.insert_many(records2)